# Multiprocessing

In [ ]:
import requests
import numpy as np

In [2]:
def load_url(url):
    # define proxyDict 
#     proxyDict = {
#         "http": ,
#         "https": ,
#     }

    resp = requests.get(url, timeout=3, stream=True, proxies=proxyDict).raw
    arr = np.asarray(bytearray(resp.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)

    return img

In [ ]:
def load_images(records):

    image_files, image_arr_list = load_urls(records['url'])

    rows = []
    images = []
    with futures.ProcessPoolExecutor(max_workers=(mp.cpu_count()-1)) as executor:
        future_to_file = {executor.submit(feature_map, im_arr, None):
                          img_file for img_file, im_arr in zip(image_files, image_arr_list)}
        for future in futures.as_completed(future_to_file):
            next_file = future_to_file[future]
            try:
                image_response = future.result()
            except Exception as e:
                print('%r generated an exception during image processing: %s' % (next_file, e))
            else:
                if image_response is None:
                    print ("no data loaded for url: {}".format(next_file))
                    continue
                else:
                    rows.append(next_file)
                    images.append(image_response[0])


    np.savez_compressed('_images.npz', features=images, records=rows)
    # data = np.load('features.npz', allow_pickle=True)
    # data['features']

# Async IO

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from load_url import fetch_all

In [ ]:
def load_urls(file_list):
    loop = asyncio.get_event_loop()  # event loop
    future = asyncio.ensure_future(fetch_all(file_list))  # could be Whitespace
    loop.run_until_complete(future)
    file_list = future.result()[0]
    image_arr_list = future.result()[1]
    return file_list, image_arr_list